In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing the required libraries

In [ ]:
import re
import nltk
nltk.download('stopwords')
from  nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### function to Preprocess the data

In [ ]:
def preprocess(data):

    porter=PorterStemmer()
    tweets=[]

    rows=len(data)
    for index in range(rows):
        tweet=re.sub("[^\w]"," ",data[index])
        tweet=tweet.lower().split()
        tweet=[porter.stem(word) for word in tweet if word not in stopwords.words('english')]
        tweet=' '.join(tweet)
        tweets.append(tweet)
    return tweets


### Reading the files

In [ ]:
train=pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isna().sum()

###  Model training

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
tweets=preprocess(train['text'])


In [ ]:
#tweets
X=cv.fit_transform(tweets).toarray()

In [ ]:
X.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,train['target'],test_size=0.33,random_state=35)

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.score(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

In [ ]:
t=preprocess(test['text'])

In [ ]:
test_X=cv.transform(t).toarray()

In [ ]:
test_X.shape

In [ ]:
output=model.predict(test_X)

In [ ]:
df=pd.DataFrame(columns=['id','target'])
df['id']=test['id']
df['target']=output

In [ ]:
df.to_csv('submission.csv',index=False,sep=',')